In [ ]:
!pip install pmdarima

     |████████████████████████████████| 1.4 MB 5.3 MB/s 
     |████████████████████████████████| 9.8 MB 34.1 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
%matplotlib inline

import numpy as np
from numpy import concatenate, percentile
import pandas as pd
from pandas import read_csv, DataFrame, concat

import matplotlib.pyplot as plt
import xgboost as xgb

from math import sqrt
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

from matplotlib import pyplot

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, TimeDistributed, RepeatVector
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from pmdarima.arima import auto_arima

In [ ]:
#Set parameters to see all data
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# Load Train dataset
train = pd.read_csv("train.csv")
train.head(5).append(train.tail(5))

,productsGroup_key,date_key,quantitySales
0,1,20190902,26784.0
1,1,20190903,7432.0
2,1,20190904,1424.0
3,1,20190905,608.0
4,1,20190906,776.0
242395,297,20211123,18.0
242396,297,20211124,16.0
242397,297,20211125,16.0
242398,297,20211126,16.0
242399,297,20211127,38.0


In [ ]:
# Load submission dataset
submission = pd.read_csv("submission.csv")
submission.head(5).append(submission.tail(5))

,productsGroup_key,date_key,quantitySales,q_0850,q_0900,q_0920,q_0950,q_0990
0,1,20211128,0,0,0,0,0,0
1,1,20211129,0,0,0,0,0,0
2,1,20211130,0,0,0,0,0,0
3,1,20211201,0,0,0,0,0,0
4,1,20211202,0,0,0,0,0,0
8311,297,20211221,0,0,0,0,0,0
8312,297,20211222,0,0,0,0,0,0
8313,297,20211223,0,0,0,0,0,0
8314,297,20211224,0,0,0,0,0,0
8315,297,20211225,0,0,0,0,0,0


In [ ]:
train['isStationary'] = 0
for product in train.productsGroup_key.unique():
  series = train[train.productsGroup_key == product].quantitySales
  X = series.values
  result = adfuller(X)
  if (result[1] > 0.05):
    train.loc[train['productsGroup_key'] == product, 'isStationary'] = 1

In [ ]:
train['isStationary'].value_counts()

0    200862
1     41538
Name: isStationary, dtype: int64

In [ ]:
stationary_train = train[train['isStationary'] == 1].copy()
non_stationary_train = train[train['isStationary'] == 0].copy()

In [ ]:
def outlier_detection(arr, isStationary):
  q25, q75 = percentile(arr, 15), percentile(arr, 85)
  iqr = q75 - q25
  if (isStationary == 1):
    # calculate the outlier cutoff
    cut_off = iqr * 1.5
  else:
    # calculate the outlier cutoff
    cut_off = iqr * 0.5
  lower, upper = q25 - cut_off, q75 + cut_off
  return lower, upper

In [ ]:
for key in stationary_train['productsGroup_key'].unique():
  lower_st, upper_st = outlier_detection(stationary_train[stationary_train['productsGroup_key'] == key]['quantitySales'], 1)

  stationary_train.loc[stationary_train['productsGroup_key'] == key, 'lower'] = lower_st
  stationary_train.loc[stationary_train['productsGroup_key'] == key, 'upper'] = upper_st

In [ ]:
for key in non_stationary_train['productsGroup_key'].unique():
  lower_nst, upper_nst = outlier_detection(non_stationary_train[non_stationary_train['productsGroup_key'] == key]['quantitySales'], 0)

  non_stationary_train.loc[non_stationary_train['productsGroup_key'] == key, 'lower'] = lower_nst
  non_stationary_train.loc[non_stationary_train['productsGroup_key'] == key, 'upper'] = upper_nst

In [ ]:
stationary_train.iloc[1]

productsGroup_key          60.00
date_key             20190902.00
quantitySales            2331.00
isStationary                1.00
lower                   -8474.85
upper                   14525.55
Name: 48315, dtype: float64

In [ ]:
stationary_train.loc[stationary_train['quantitySales'] < stationary_train['lower'], 'quantitySales'] = stationary_train['lower']
stationary_train.loc[stationary_train['quantitySales'] > stationary_train['upper'], 'quantitySales'] = stationary_train['upper']
non_stationary_train.loc[non_stationary_train['quantitySales'] < non_stationary_train['lower'], 'quantitySales'] = non_stationary_train['lower']
non_stationary_train.loc[non_stationary_train['quantitySales'] > non_stationary_train['upper'], 'quantitySales'] = non_stationary_train['upper']


In [ ]:
stationary_train.shape, non_stationary_train.shape

((41538, 6), (200862, 6))

In [ ]:
train_df = stationary_train.append(non_stationary_train)
train_df.shape

(242400, 6)

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        # print(interval, dataset[i], dataset[i - interval])
        diff.append(value)
    return pd.Series(diff)

In [ ]:
train_df.head()

,productsGroup_key,date_key,quantitySales,isStationary,lower,upper
48314,60,20190901,153.0,1,-8474.85,14525.55
48315,60,20190902,2331.0,1,-8474.85,14525.55
48316,60,20190903,2781.0,1,-8474.85,14525.55
48317,60,20190904,3744.0,1,-8474.85,14525.55
48318,60,20190905,3078.0,1,-8474.85,14525.55


In [ ]:
train_df.index = pd.to_datetime(train_df.date_key)
train_df.head()

,productsGroup_key,date_key,quantitySales,isStationary,lower,upper
date_key,,,,,,
1970-01-01 00:00:00.020190901,60,20190901,153.0,1,-8474.85,14525.55
1970-01-01 00:00:00.020190902,60,20190902,2331.0,1,-8474.85,14525.55
1970-01-01 00:00:00.020190903,60,20190903,2781.0,1,-8474.85,14525.55
1970-01-01 00:00:00.020190904,60,20190904,3744.0,1,-8474.85,14525.55
1970-01-01 00:00:00.020190905,60,20190905,3078.0,1,-8474.85,14525.55


In [ ]:
def split_data_ratio(df, train, submission):
  n_train = train.shape[0] / (train.shape[0] + submission.shape[0])
  n_train = round(df.shape[0] * n_train)
  return n_train

In [ ]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    dfx = pd.DataFrame(data)
    df = dfx.assign(**{
        '{} (t-{})'.format(col, t): dfx[col].shift(t)
        for t in range(lag+1)
        for col in dfx
    })

    df=df.drop([df.columns[0]], axis=1)
    df=df[df.columns[::-1]]
    return df[lag:]

In [ ]:
# scale train and test data to [-1, 1] with MinMaxScaler
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler()  
    
    # StandardScaler()
    scaler = scaler.fit(train)

    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    tr_scaled = scaler.transform(train)

    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    ts_scaled = scaler.transform(test)
    
    return scaler, tr_scaled, ts_scaled

In [ ]:
class model_engine:
  def __init__(self, df, productsGroup_key):
    self.df = df
    self.productsGroup_key = productsGroup_key
  
  def xgb_regressor(train_df, train_X, train_y, test_scaled, scaler, n_train):
    # XGBRegressor Training
    now = datetime.now()
    print("Process started at : ",now)

    parameters = { 'gamma' : [0, 0.1, 0.3, 1], 'learning_rate' : [0.001, 0.01, 0.1], 
                  'max_depth' : [2, 4, 6, 7, 12], 
                  'n_estimators' : [10, 45, 90, 100, 150, 250],
                  'nthread' : [-1], 'reg_alpha' : [1], 'reg_lambda' : [1], 'seed' : [10] }

    bst = xgb.XGBRegressor()
    xgb_grid = GridSearchCV(bst,
                            parameters,
                            cv=5,
                            n_jobs=-1,
                            verbose=True,
                            )
    xgb_grid.fit(train_X, train_y, eval_set=[(train_X, train_y)], early_stopping_rounds=50)

    end = datetime.now()
    print("Process finished at : ", end)
    print("Process took : ", end-now)

    # inverse scaling for a forecasted value
    def invert_scale(scaler, X, value):
        new_row = [x for x in X] + [value]
        array = np.array(new_row)
        array = array.reshape(1, len(array))

        inverted = scaler.inverse_transform(array)
        ##print("converting %s to %s" % (value,inverted[0, -1]))
        return inverted[0, -1]
    # invert differenced value
    def inverse_difference(history, yhat, interval=1):
      return yhat + history[-interval]

    def mean_absolute_percentage_error(y_true, y_pred):
      return np.mean(np.abs((np.array(y_true) - np.array(y_pred)) / y_true)) * 100

    #Invert scale predictions to time series
    predictions = list()
    start = 28 # test period
    l = len(test_scaled) - start
    print(l)
    rmse = []
    mape = []

    for i in range(len(test_scaled)):
        X1, y = test_scaled[l, 0:-1], test_scaled[l, -1]
        X1 = X1[-30:] # ts_window

        X1 = X1.reshape((1,-1))
        pred = xgb_grid.predict(X1)

        yhat = invert_scale(scaler, X1[0], pred)
            
        l=l+1

        yhat = inverse_difference(train_df2['quantitySales'], yhat, len(test_scaled) + 1 - i) # if not stationary 

        predictions.append(yhat)

        rmse.append(sqrt(mean_squared_error([train_df2['quantitySales'][n_train:][i]], [yhat])))
        mape.append(mean_absolute_percentage_error([train_df2['quantitySales'][n_train:][i]], [yhat]))

    print("Test RMSE:", np.mean(rmse))
    print("Test MAPE:", np.mean(mape))

    return predictions

  def auto_arima(train_df2):

    train_df_arima = train_df2.drop(columns = ['productsGroup_key', 'date_key', 'isStationary', 'lower', 'upper'], axis=1)

    arima_train = train_df_arima[:-28]
    arima_test = train[-28:]

    # Arima Training
    now = datetime.now()
    print("Arima process started at : ",now)

    arima_stepwise_model = auto_arima(train_df_arima, start_p=0, start_q=0,
                              max_p=13, max_q=13, m=12,
                              start_P=0, seasonal=True,
                              d=1, D=1, trace=True,
                              error_action='ignore',  
                              suppress_warnings=True, 
                              stepwise=True,
                              n_jobs=4)#n_jobs for parallel process
    end = datetime.now()

    print("Arima process finished at : ", end)
    print("Process took : ", end-now)

    print(arima_stepwise_model)

    # Fit arima model
    arima_predicts = arima_stepwise_model.fit(arima_train)

    # Predict results with auto-arima
    future_forecast = arima_predicts.predict(n_periods=28)

    #arima_rmse = sqrt(mean_squared_error(train_df_arima['quantitySales'][-26:], future_forecast))
    #arima_mape = mean_absolute_percentage_error(train_df_arima['quantitySales'][-26:], future_forecast)

    #print("Test RMSE:", np.mean(arima_rmse))
    #print("Test MAPE:", np.mean(arima_mape))

    return future_forecast

  def LSTM(train_df2, train_X_lstm, train_Y_lstm, test_X_lstm, test_scaled, scaler, n_train):

    # define parameters
    verbose, epochs, batch_size = 1, 25, 50
    n_timesteps, n_features, n_outputs = train_X_lstm.shape[1], train_X_lstm.shape[2], train_Y_lstm.shape[1]

    # define model
    lstm_model = Sequential()
    lstm_model.add(Conv1D(filters=32, kernel_size=5,
                          strides=1, padding="causal",
                          activation="relu",
                          input_shape=(n_timesteps, n_features)))
    
    lstm_model.add(LSTM(500, activation='relu', return_sequences=True))
    lstm_model.add(LSTM(500, activation='relu', return_sequences=True))
    lstm_model.add(TimeDistributed(Dense(1, activation='relu')))
    lstm_model.compile(loss='mae', optimizer=Adam(lr=0.001),
                       metrics=['acc'])
    
    now = datetime.now()

    history = lstm_model.fit(train_X_lstm, train_Y_lstm, epochs=epochs, batch_size=batch_size, validation_data=(test_X_lstm),
                             callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=verbose, shuffle=False)

    lstm_model.summary()

    end = datetime.now()
    print("LSTM process finished at : ", end)
    print("LSTM process took : ", end-now)

    # Predict results with LSTM Model
    lstm_predicts = lstm_model.predict(test_X_lstm, verbose=verbose)

    # Invert scaled predictons to time series
    predictions = list()
    start = 28 # test period
    l = len(test_scaled) - start

    lstm_rmse = []
    lstm_mape = []

    for i in range(len(test_scaled)):
        X1, y = test_scaled[l, 0:-1], test_scaled[l, -1]
        X1 = X1[-30:] # ts_window

        X1 = X1.reshape((1,-1))

        yhat = invert_scale(scaler, X1[0], lstm_predicts[i][0])
            
        l=l+1

        yhat = inverse_difference(train_df2['quantitySales'], yhat, len(test_scaled) + 1 - i)

        predictions.append(yhat)

        #lstm_rmse.append(sqrt(mean_squared_error([train_df2['quantitySales'][n_train:][i]], [yhat])))
        #lstm_mape.append(mean_absolute_percentage_error([train_df2['quantitySales'][n_train:][i]], [yhat]))
    
    #print("LSTM RMSE:", np.mean(lstm_rmse))
    #print("LSTM MAPE:", np.mean(lstm_mape))

    return predictions



In [ ]:
window_size = 30
#for key in train_df['productsGroup_key'].unique():
key_list = [1,2,3]
for key in key_list:

  train_df2 = train_df[train_df.productsGroup_key == key]
  diff_values = difference(train_df2.quantitySales, 1)

  n_train = split_data_ratio(train_df2, train, submission)

  # split train test datasets
  #xtrain, xtest = diff_values[0:n_train], diff_values[n_train:]
  xtrain = diff_values
  xtest = submission.quantitySales

  # reorganize dataset acording to window size
  #values_unscaled = np.concatenate((xtrain, xtest))
  values_unscaled = xtrain
  supervised_raw = timeseries_to_supervised(values_unscaled, window_size)

  #Convert value type as float32
  supervised_raw = supervised_raw.values.astype("float32")

  #Scale values
  scaler, train_scaled, test_scaled = scale(xtrain.values.reshape(len(xtrain), 1), xtest.values.reshape(len(xtest), 1))

  #values_scaled = np.concatenate((train_scaled, test_scaled))
  supervised_train = timeseries_to_supervised(train_scaled, window_size)
  supervised_test = timeseries_to_supervised(test_scaled, window_size)


  supervised_values = supervised_train.values.astype('float32')

  #Split supervised data into train and test sets
  #supervised_train, supervised_test = supervised_raw[0:n_train], supervised_raw[n_train:]

  #Split scaled data into train and test-sets
  train_scaled, test_scaled = supervised_values, supervised_test

  #Split scaled data into X and y
  train_X, train_y = train_scaled[:, :-1], train_scaled[:, -1]
  #test_X, test_y = test_scaled[:, :-1], test_scaled[:, -1]

  #Reshape train_y
  train_y.reshape(train_y.shape[0], 1)

  # prepare train dataset for lstm
  train_X_lstm = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
  test_X_lstm = test_scaled.quantitySales.reshape((test_scaled.shape[0], 1, test_scaled.shape[1]))

  # prepare test dataset for lstm
  train_Y_lstm = train_y.reshape((train_y.shape[0], 1, 1))
  #test_Y_lstm = test_y.reshape((test_y.shape[0], 1, 1))


  ##########################################################
  ################### ***XGBRegressor*** ###################
  ##########################################################

  # XGBRegressor Training
  
  models = model_engine() 

  result_df['XGB_Predictions'] = models.xgb_regressor(train_df2, train_X, train_y, test_scaled, scaler, n_train)

  result.append(result_df)

  ##########################################################
  #################### ***AutoArima*** #####################
  ##########################################################
  
  
  result_df['Arima_Predictions'] = models.auto_arima(train_df2)
  result.append(result_df)

  ##########################################################
  ####################### ***LSTM*** #######################
  ##########################################################

  result_df['LSTM_Predictions'] = models.LSTM(train_df2, train_X_lstm, train_Y_lstm, test_X_lstm, test_scaled, scaler, n_train)
  result.append(result_df)

  result.to_csv("result.csv")